In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

from models.nested_validation import *
from models.features import fixed_makedata
from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time, auto_save_pickle


def item2df(item):
    name, pickle_path = item
    results = load_pickle(pickle_path)
    mean_score = average_score(results)
    df = pd.DataFrame.from_dict(mean_score, orient = 'index').mean(axis = 0)
    df = round(df,3)
    df['method'] = results[0]['method']
    df['name'] = name
    df = df.to_frame().T
    return df

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [2]:
from utils.kw_dunn import kw_dunn
from itertools import combinations
from scipy import stats

def aggre(results, metric = 'f1'):
    return [results[i]['test_'+metric] for i in range(5)]

def test(dic_results, metric = 'f1'):
    f1s = {k: aggre(load_pickle(pickle_path), metric) for k, pickle_path in dic_results.items()}
    gb = pd.Series(f1s)
    H, p_omnibus, Z_pairs, p_corrected, reject = kw_dunn(gb)
    print(('p_omnibus',p_omnibus))
    names = tuple(gb.index.tolist())
    for pair, r, p in tuple(zip(list(combinations(names,2)),reject,p_corrected)):
        if r:
            print(pair,r,p)
    
def mtest(dic_results):
    f1s = {k: aggre_f1(results) for k, results in dic_results.items()}
    gb = pd.Series(f1s)
    names = tuple(gb.index.tolist())
    print(stats.mannwhitneyu(f1s[names[0]],f1s[names[1]]))


time: 31 ms


In [3]:
dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
dic = load(dir_save)

amt_GI = load_pickle(dir_HugeFiles+'dph/amt_GI_20190701.pickle')
for i, v in dic.items():
    if i in amt_GI.keys():
        v['AMT'] = amt_GI[i]
amt_ls = list(amt_GI.keys())
ls = amt_ls

exist
time: 9.65 s


In [4]:
table4_LR = {
    'BoW-basic':pd.read_csv('../csv/2019-08-20 16:18:42.533625.csv').pickle_path[0],
    'BoW-parsed':pd.read_csv('../csv/2019-08-20 16:49:46.286243.csv').pickle_path[0],
    'Bow-NB':pd.read_csv('../csv/2019-08-20 15:52:26.255971.csv').pickle_path[0],
    'Pretrained-word2vec':pd.read_csv('../csv/2019-08-21 08:17:40.580092.csv').pickle_path[1],
    'Pretrained-glove':pd.read_csv('../csv/2019-08-21 08:17:40.580092.csv').pickle_path[2],
    'Pretrained-fasttext':pd.read_csv('../csv/2019-08-21 08:17:40.580092.csv').pickle_path[3],  
    'Recipe54k-word2vec':pd.read_csv('../csv/2019-08-20 20:17:17.822381.csv').pickle_path[0],
    'Recipe54k-glove':pd.read_csv('../csv/2019-08-21 11:45:17.446046.csv').pickle_path[0],
    'Recipe54k-fasttext':pd.read_csv('../csv/2019-08-21 11:36:12.417320.csv').pickle_path[0],
    'Recipe1M-word2vec':pd.read_csv('../csv/2019-08-21 08:17:40.580092.csv').pickle_path[0],
    'Sentence-skipthoughts':pd.read_csv('../csv/2019-08-23 16:18:27.648576.csv').pickle_path[0],
    'Document-doc2vec':pd.read_csv('../csv/2019-08-21 14:23:43.682872.csv').pickle_path[0],
}
             
table4_LGBM = {
    'BoW-basic':pd.read_csv('../csv/2019-08-21 08:48:00.938174.csv').pickle_path[0],
    'BoW-parsed':pd.read_csv('../csv/2019-08-21 07:20:43.162296.csv').pickle_path[0],
    'BoW-NB':pd.read_csv('../csv/2019-08-21 23:42:10.497253.csv').pickle_path[0],
    'Pretrained-word2vec':pd.read_csv('../csv/2019-08-22 22:35:05.475816.csv').pickle_path[0],
    'Pretrained-glove':pd.read_csv('../csv/2019-08-23 10:52:21.004801.csv').pickle_path[0],
    'Pretrained-fasttext':pd.read_csv('../csv/2019-08-23 20:18:18.566324.csv').pickle_path[0],  
    'Recipe54k-word2vec':pd.read_csv('../csv/2019-08-22 10:52:20.164045.csv').pickle_path[0],
    'Recipe54k-glove':pd.read_csv('../csv/2019-08-22 08:52:54.812613.csv').pickle_path[0],
    'Recipe54k-fasttext':pd.read_csv('../csv/2019-08-22 11:20:03.538590.csv').pickle_path[1],
    'Recipe1M-word2vec':pd.read_csv('../csv/2019-08-22 08:45:01.716825.csv').pickle_path[0],
    'Sentence-skipthoughts':pd.read_csv('../csv/2019-08-24 11:24:28.152424.csv').pickle_path[0],
    'Document-doc2vec':pd.read_csv('../csv/2019-08-22 11:09:36.565984.csv').pickle_path[0],
}       

time: 137 ms


In [5]:
display(pd.concat([item2df(items) for items in table4_LR.items()]))
display(pd.concat([item2df(items) for items in table4_LGBM.items()]))


test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.76    0.81           0.717       0.769         0.997    1                
0  0.764   0.798          0.733       0.768         0.997    0.999            
0  0.817   0.844          0.794       0.818         0.938    0.948            
0  0.779   0.793          0.771       0.777         0.857    0.876            
0  0.784   0.789          0.784       0.78          0.874    0.876            
0  0.78    0.777          0.785       0.775         0.872    0.877            
0  0.78    0.787          0.778       0.776         0.865    0.883            
0  0.783   0.783          0.788       0.778         0.868    0.864            
0  0.787   0.778          0.8         0.779         0.887    0.894            
0  0.815   0.815          0.816       0.81          0.905    0.918            
0  0.79    0.803          0.779       0.788         0.941    0.949            
0  0.745   0.761          0.731       0.744         0.803    0.816            

  train_recall train_accuracy         method                   name  
0  0.994        0.997          [wc, scale]    BoW-basic              
0  0.996        0.997          [wc, scale]    BoW-parsed             
0  0.929        0.938          [wc, nb, 12]   Bow-NB                 
0  0.84         0.857          [wv, scale]    Pretrained-word2vec    
0  0.872        0.871          [wv, scale]    Pretrained-glove       
0  0.868        0.87           [wv, scale]    Pretrained-fasttext    
0  0.849        0.865          [wv, scale]    Recipe54k-word2vec     
0  0.873        0.865          [wv, scale]    Recipe54k-glove        
0  0.88         0.885          [wv, scale]    Recipe54k-fasttext     
0  0.893        0.904          [wv, scale]    Recipe1M-word2vec      
0  0.933        0.94           [skip, scale]  Sentence-skipthoughts  
0  0.79         0.802          [wv, scale]    Document-doc2vec

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.76    0.77           0.751       0.759         1        1                
0  0.755   0.757          0.753       0.749         1        1                
0  0.77    0.774          0.767       0.767         1        1                
0  0.774   0.77           0.779       0.767         1        1                
0  0.802   0.814          0.791       0.8           1        1                
0  0.761   0.779          0.749       0.761         1        1                
0  0.779   0.78           0.781       0.775         1        1                
0  0.783   0.793          0.775       0.781         1        1                
0  0.755   0.76           0.753       0.75          1        1                
0  0.787   0.823          0.759       0.791         1        1                
0  0.79    0.818          0.767       0.793         1        1                
0  0.763   0.78           0.749       0.762         1        1                

  train_recall train_accuracy         method                   name  
0  1            1              [wc, scale]    BoW-basic              
0  1            1              [wc, scale]    BoW-parsed             
0  1            1              [wc, nb]       BoW-NB                 
0  1            0.999          [wv, scale]    Pretrained-word2vec    
0  1            1              [wv, scale]    Pretrained-glove       
0  1            1              [wv, scale]    Pretrained-fasttext    
0  1            1              [wv, scale]    Recipe54k-word2vec     
0  1            1              [wv, scale]    Recipe54k-glove        
0  1            0.999          [wv, scale]    Recipe54k-fasttext     
0  1            1              [wv, scale]    Recipe1M-word2vec      
0  1            1              [skip, scale]  Sentence-skipthoughts  
0  1            0.999          [wv, scale]    Document-doc2vec

time: 17.8 s


In [6]:
table5_LR_NU = {
    'Nutritions':pd.read_csv('../csv/2019-08-22 14:30:58.433842.csv').pickle_path[0],
    'BoW-NB, Nutritions':pd.read_csv('../csv/2019-08-22 11:00:46.010297.csv').pickle_path[0],
    'Recipe1M-word2vec, Nutritions':pd.read_csv('../csv/2019-08-22 13:17:51.399916.csv').pickle_path[0],
    'Sentence-skipthoughts, Nutritions':pd.read_csv('../csv/2019-08-23 16:46:04.623991.csv').pickle_path[0],
    'BoW-NB, Recipe1M-word2vec, Nutritions':pd.read_csv('../csv/2019-08-22 13:58:58.023957.csv').pickle_path[0],
    'BoW-NB, Recipe1M-word2vec, Sentence-skipthoughts, Nutritions':pd.read_csv('../csv/2019-08-23 17:41:26.704592.csv').pickle_path[0],
    
}
display(pd.concat([item2df(items) for items in table5_LR_NU.items()]))

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.825   0.83           0.822       0.822         0.839    0.841            
0  0.85    0.859          0.842       0.847         0.933    0.937            
0  0.836   0.851          0.824       0.834         0.906    0.923            
0  0.821   0.857          0.79        0.824         0.931    0.956            
0  0.849   0.861          0.842       0.847         0.933    0.939            
0  0.842   0.86           0.826       0.841         0.953    0.964            

  train_recall train_accuracy                         method  \
0  0.837        0.836          [nu, scale]                     
0  0.929        0.932          [wc, nb, nu, scale]             
0  0.889        0.906          [wv, nu, scale]                 
0  0.908        0.932          [skip, nu, scale]               
0  0.928        0.933          [wc, nb, wv, nu, scale]         
0  0.942        0.953          [wc, nb, wv, nu, skip, scale]   

                                                           name  
0  Nutritions                                                    
0  BoW-NB, Nutritions                                            
0  Recipe1M-word2vec, Nutritions                                 
0  Sentence-skipthoughts, Nutritions                             
0  BoW-NB, Recipe1M-word2vec, Nutritions                         
0  BoW-NB, Recipe1M-word2vec, Sentence-skipthoughts, Nutritions

time: 1.04 s


In [7]:
table5_LGBM_NU = {
    'Nutritions':pd.read_csv('../csv/2019-08-24 10:37:51.778201.csv').pickle_path[0],
    'BoW-NB, Nutritions':pd.read_csv('../csv/2019-08-22 22:58:47.373844.csv').pickle_path[0],
    'glove-pretrained, Nutritions':pd.read_csv('../csv/2019-08-24 09:17:21.691140.csv').pickle_path[0],
    'Sentence-skipthoughts, Nutritions':pd.read_csv('../csv/2019-08-24 14:23:55.087730.csv').pickle_path[0],
}
display(pd.concat([item2df(items) for items in table5_LGBM_NU.items()]))

test_f1 test_precision test_recall test_accuracy train_f1 train_precision  \
0  0.811   0.81           0.812       0.806         1        1                
0  0.831   0.823          0.84        0.825         1        1                
0  0.854   0.852          0.858       0.85          1        1                
0  0.843   0.847          0.84        0.839         1        1                

  train_recall train_accuracy               method  \
0  1            1              [nu, scale]           
0  1            1              [wc, nb, nu, scale]   
0  1            1              [wv, nu, scale]       
0  1            1              [skip, nu, scale]     

                                name  
0  Nutritions                         
0  BoW-NB, Nutritions                 
0  glove-pretrained, Nutritions       
0  Sentence-skipthoughts, Nutritions

time: 2.18 s


In [8]:
tableLR = table4_LR
tableLR.update(table5_LR_NU)
test(tableLR, metric = 'f1')

('p_omnibus', 2.925574153211479e-07)
('BoW-NB, Nutritions', 'Document-doc2vec') True 0.002952004628755212
('BoW-NB, Recipe1M-word2vec, Nutritions', 'Document-doc2vec') True 0.0038661555813195208
('BoW-NB, Recipe1M-word2vec, Sentence-skipthoughts, Nutritions', 'Document-doc2vec') True 0.005756638824387029
('Document-doc2vec', 'Recipe1M-word2vec, Nutritions') True 0.019044555800660418
time: 5.98 s


In [9]:
tableLGBM = table4_LGBM
tableLGBM.update(table5_LGBM_NU)
test(tableLGBM, metric = 'f1')

('p_omnibus', 0.0001037716026943916)
('BoW-parsed', 'glove-pretrained, Nutritions') True 0.04474650709093663
('Recipe54k-fasttext', 'Sentence-skipthoughts, Nutritions') True 0.04712143758709969
('Recipe54k-fasttext', 'glove-pretrained, Nutritions') True 0.019629575042546664
time: 12.6 s


### Feature importance

In [10]:
LR_NU = load_pickle(table5_LR_NU['Nutritions'])
grouped = pd.concat([result['f_importance'] for result in LR_NU]).groupby(['feature']).mean().unstack().unstack()
grouped.columns = [col_name[:-11] if col_name != 'dry_weight' else col_name for col_name in grouped.columns]
grouped = grouped.T
display(grouped.sort_values(by = ['importance'], ascending = True))
display(grouped.sort_values(by = ['importance'], ascending = False))

importance
protein             -0.676497  
total fat           -0.544760  
dietary fiber       -0.540488  
potassium           -0.527181  
sugars              -0.270326  
sodium              -0.251458  
calcium             -0.077129  
iron                -0.058733  
vitamin c           -0.013003  
thiamin              0.009224  
calorie              0.016746  
folate               0.022545  
vitamin b6           0.027560  
vitamin a            0.069214  
cholesterol          0.109013  
niacin               0.116694  
magnesium            0.146575  
saturated fat        0.197333  
dry_weight           0.430556  
total carbohydrates  1.236239

importance
total carbohydrates  1.236239  
dry_weight           0.430556  
saturated fat        0.197333  
magnesium            0.146575  
niacin               0.116694  
cholesterol          0.109013  
vitamin a            0.069214  
vitamin b6           0.027560  
folate               0.022545  
calorie              0.016746  
thiamin              0.009224  
vitamin c           -0.013003  
iron                -0.058733  
calcium             -0.077129  
sodium              -0.251458  
sugars              -0.270326  
potassium           -0.527181  
dietary fiber       -0.540488  
total fat           -0.544760  
protein             -0.676497

time: 56.8 ms


### Display the NB weights; Here we showed the NB calculated from the whole dataset
Note: In the actual experiment, we calculated the NB weight from training set only

In [11]:
corp0 = fixed_makedata(dic, ls, tag='AMT')

time: 7.6 s


In [12]:
X, feature_name = get_wordcount_list(corp0.corpus_list,  higherthan = 0)  
y_pos, y_neg = corp0.y.astype(bool), ~corp0.y.astype(bool)
alpha = 1
p = X[y_pos].sum(axis = 0) + alpha
sum_p = p.sum() + alpha
q = X[y_neg].sum(axis = 0) + alpha
sum_q = q.sum() + alpha
count_ratio = np.log(p/sum_p) - np.log(q/sum_q)

time: 758 ms


In [13]:
df_NB = pd.DataFrame.from_dict(dict(zip(feature_name,count_ratio)),orient= 'index')
display(df_NB.sort_values(by = [0]).head(20))
display(df_NB.sort_values(by = [0], ascending = False).head(20))

0
steaks     -3.834391
rub        -3.141244
swordfish  -3.041161
nonfat     -2.804772
brussels   -2.735779
oysters    -2.661671
quail      -2.661671
tails      -2.494617
marjoram   -2.494617
duck       -2.399307
liver      -2.399307
grate      -2.315925
fennel     -2.293946
broiling   -2.293946
ancho      -2.176163
freekeh    -2.176163
sauerkraut -2.176163
korean     -2.042632
containers -2.042632
daikon     -2.042632

0
linguine    3.566840
balls       3.514196
pumpkin     3.429639
yeast       3.369014
vodka       2.994321
cake        2.899011
cereal      2.899011
rye         2.847717
chocolate   2.793650
shaker      2.675867
watermelon  2.675867
al          2.630197
dente       2.611329
walnuts     2.542336
cookies     2.542336
gin         2.542336
fettuccini  2.542336
pans        2.542336
rotini      2.542336
fettuccine  2.542336

time: 36.9 ms
